In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula_middle_padding import Tabula 
import pandas as pd
import torch

In [2]:
data = pd.read_csv("Real_Datasets/cleaned_credit_score_v2.csv")
data = data.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan', 'Payment_Behaviour'], axis=1)

In [3]:
#categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
model = Tabula(llm='distilgpt2', experiment_dir = "credit-score_training", batch_size=32, epochs=128)

## In following block, it is important to indicate "conditional_col = data.columns[0]". Otherwise, the generation will use target column as conditional column, which will result in error!

In [4]:
model.fit(data, conditional_col = data.columns[0])

/home/svenlag/tabula_test/Tabula/tabula_middle_padding/tabula.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.810200
1000,2.321700
1500,2.272100
2000,2.239000
2500,2.210200
3000,2.195300
3500,2.176600
4000,2.165000
4500,2.146200
5000,2.130900


In [ ]:
import torch
torch.save(model.model.state_dict(), "credit-score_training/model_128epoch_middle_padding.pt")

In [5]:
torch.save(model, "credit-score_training/model_128epoch_middle_padding_FULL.pt")

In [3]:
# Specify weights_only=False to allow full unpickling
model = torch.load(
    "credit-score_training/model_128epoch_middle_padding_FULL.pt",
    map_location=torch.device("cuda"),      # or "cpu" as needed
    weights_only=False
)
model.model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
for i in range(9,31):
    synthetic_data = model.sample(n_samples=33769)
    synthetic_data.to_csv(f"credit-score_sd-sets/credit-score_128epoch_middle_padding_{i}.csv", index=False)

  0%|          | 0/33769 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
33799it [06:46, 83.15it/s]                           
33799it [06:56, 81.06it/s]                           
33800it [06:57, 80.98it/s]                           
33800it [06:59, 80.52it/s]                           
33800it [07:00, 80.35it/s]                           
33800it [07:00, 80.36it/s]                           
33799it [07:00, 80.35it/s]                           
33799it [07:00, 80.46it/s]                           
33800it [07:02, 80.07it/s]                           
33799it [07:00, 80.47it/s]                           
33798it [07:02, 80.04it/s]                           
33800it [07:00, 80.36it/s]                           
33800it [07:00, 80.29it/s]                           
33800it [07:01, 80.15it/s]  

In [ ]:
synthetic_data = model.sample(n_samples=33769)
synthetic_data.to_csv("credit-score_128epoch_middle_padding_1.csv", index=False)

In [7]:
from sdmetrics.reports.single_table import QualityReport

synthetic_data = synthetic_data[data.columns.to_list()]

metadata = {
    "columns": {
        "Month": {"sdtype": "categorical"},
        "Age": {"sdtype": "numerical"},
        "Occupation": {"sdtype": "categorical"},
        "Annual_Income": {"sdtype": "numerical"},
        "Monthly_Inhand_Salary": {"sdtype": "numerical"},
        "Num_Bank_Accounts": {"sdtype": "numerical"},
        "Num_Credit_Card": {"sdtype": "numerical"},
        "Interest_Rate": {"sdtype": "numerical"},
        "Num_of_Loan": {"sdtype": "numerical"},
        "Delay_from_due_date": {"sdtype": "numerical"},
        "Num_of_Delayed_Payment": {"sdtype": "numerical"},
        "Changed_Credit_Limit": {"sdtype": "numerical"},
        "Num_Credit_Inquiries": {"sdtype": "numerical"},
        "Credit_Mix": {"sdtype": "categorical"},
        "Outstanding_Debt": {"sdtype": "numerical"},
        "Credit_Utilization_Ratio": {"sdtype": "numerical"},
        "Credit_History_Age": {"sdtype": "numerical"},
        "Payment_of_Min_Amount": {"sdtype": "categorical"},
        "Total_EMI_per_month": {"sdtype": "numerical"},
        "Amount_invested_monthly": {"sdtype": "numerical"},
        "Monthly_Balance": {"sdtype": "numerical"},
    }
}

report = QualityReport()
report.generate(data, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 21/21 [00:00<00:00, 105.31it/s]|
Column Shapes Score: 94.76%

(2/2) Evaluating Column Pair Trends: |██████████| 210/210 [00:01<00:00, 120.00it/s]|
Column Pair Trends Score: 75.33%

Overall Score (Average): 85.05%

